# LDA Training
<figure>
<img src=https://s2.loli.net/2022/02/28/X7vzOlDHJtP6UnM.png style="width: 600px">
<figcaption>The LDA training algorithm from <a href=http://www.arbylon.net/publications/text-est.pdf>Parameter estimation for text analysis</a></figcaption>
</figure>

In [2]:
import random
import numpy as np
from collections import defaultdict, OrderedDict
from types import SimpleNamespace
from tqdm.notebook import tqdm
from visualize import visualize_topic_word

In [3]:
# === corpus loading ===
class NeurIPSCorpus:
    def __init__(self, data_path, num_topics, max_num_docs=100, max_num_words=10000, max_doc_length=1000):
        self.docs = []
        self.word2id = OrderedDict()
        self.max_doc_length = max_doc_length

        # only keep the most frequent words
        word2cnt = defaultdict(int)
        with open(data_path) as fin:
            for i, line in enumerate(list(fin)[::-1]):  # use more recent papers
                if i >= max_num_docs: break
                for word in line.strip().split():
                    word2cnt[word] += 1
        
        word2cnt = sorted(list(word2cnt.items()), key=lambda x: x[1], reverse=True)
        if len(word2cnt) > max_num_words:
            word2cnt = word2cnt[:max_num_words]
        word2cnt = dict(word2cnt)

        # read in the doc and convert words to integers
        with open(data_path) as fin:
            for i, line in enumerate(list(fin)[::-1]):  # use more recent papers
                if i >= max_num_docs: break
                doc = []
                for word in line.strip().split():
                    if len(doc) >= self.max_doc_length: break
                    if word not in word2cnt: continue
                    if word not in self.word2id: 
                        self.word2id[word] = len(self.word2id)
                    doc.append(self.word2id[word])
                self.docs.append(doc)

        self.num_docs = len(self.docs)
        self.num_topics = num_topics
        self.num_words = len(self.word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}
        print(
            "num_docs:", self.num_docs, 
            "num_topics:", self.num_topics, 
            "num_words:", self.num_words
        )

corpus = NeurIPSCorpus(
    data_path="data/papers.txt", 
    num_topics=10,
    max_num_docs=1000,
    max_num_words=10000,
    max_doc_length=200,
)
hparams = SimpleNamespace(
    alpha=np.ones([corpus.num_topics], dtype=float) / corpus.num_topics,
    beta = np.ones([corpus.num_words], dtype=float) / corpus.num_topics,
    gibbs_sampling_max_iters=500,
)

num_docs: 1000 num_topics: 10 num_words: 7794


In [4]:
# === initialization ===
print("Initializing...", flush=True)
n_doc_topic = np.zeros([corpus.num_docs, corpus.num_topics], dtype=float) # n_m^(k)
n_topic_word = np.zeros([corpus.num_topics, corpus.num_words], dtype=float) # n_k^(t)
z_doc_word = np.zeros([corpus.num_docs, corpus.max_doc_length], dtype=int)

for doc_i in range(corpus.num_docs):
    for j, word_j in enumerate(corpus.docs[doc_i]):
        topic_ij = random.randint(0, corpus.num_topics - 1)
        n_doc_topic[doc_i, topic_ij] += 1
        n_topic_word[topic_ij, word_j] += 1
        z_doc_word[doc_i, j] = topic_ij

# === Gibbs sampling ===
print("Gibbs sampling...", flush=True)
for iteration in tqdm(range(hparams.gibbs_sampling_max_iters)):
    for doc_i in range(corpus.num_docs):
        for j, word_j in enumerate(corpus.docs[doc_i]):
            # remove the old assignment
            topic_ij = z_doc_word[doc_i, j]
            n_doc_topic[doc_i, topic_ij] -= 1
            n_topic_word[topic_ij, word_j] -= 1
            # compute the new assignment
            p_doc_topic = (n_doc_topic[doc_i, :] + hparams.alpha) \
                        / np.sum(n_doc_topic[doc_i] + hparams.alpha)
            p_topic_word = (n_topic_word[:, word_j] + hparams.beta[word_j]) \
                        / np.sum(n_topic_word + hparams.beta, axis=1)
            p_topic = p_doc_topic * p_topic_word
            p_topic /= np.sum(p_topic)
            # record the new assignment
            new_topic_ij = np.random.choice(np.arange(corpus.num_topics), p=p_topic)
            n_doc_topic[doc_i, new_topic_ij] += 1
            n_topic_word[new_topic_ij, word_j] += 1
            z_doc_word[doc_i, j] = new_topic_ij

    if iteration % 50 == 0:
        print(f"Iter [{iteration}]===")
        # === Check convergence and read out parameters ===
        theta = (n_doc_topic + hparams.alpha) / np.sum(n_doc_topic + hparams.alpha, axis=1, keepdims=True)
        phi = (n_topic_word + hparams.beta) / np.sum(n_topic_word + hparams.beta, axis=1, keepdims=True)

        all_top_words = []
        all_top_probs = []
        for topic in range(corpus.num_topics):
            top_words = np.argsort(phi[topic])[::-1][:10]
            top_probs = phi[topic, top_words]
            top_words = [corpus.id2word[word] for word in top_words]
            all_top_words.append(top_words)
            all_top_probs.append(top_probs)
            print(f"Topic {topic}:", top_words)
        visualize_topic_word(all_top_words, all_top_probs)

Initializing...
Gibbs sampling...


Iter [0]===
Topic 0: ['model', 'network', 'learning', 'problem', 'learn', 'function', 'algorithm', 'university', 'datum', 'method']
Topic 1: ['network', 'learning', 'model', 'problem', 'learn', 'method', 'algorithm', 'datum', 'introduction', 'task']
Topic 2: ['model', 'network', 'learning', 'datum', 'problem', 'algorithm', 'neural', 'university', 'edu', 'function']
Topic 3: ['model', 'problem', 'learn', 'network', 'learning', 'university', 'algorithm', 'abstract', 'datum', 'edu']
Topic 4: ['model', 'network', 'learning', 'problem', 'university', 'algorithm', 'learn', 'abstract', 'function', 'method']
Topic 5: ['model', 'neural', 'network', 'function', 'algorithm', 'problem', 'learning', 'learn', 'university', 'edu']
Topic 6: ['model', 'datum', 'network', 'neural', 'learning', 'method', 'university', 'learn', 'algorithm', 'problem']
Topic 7: ['network', 'model', 'algorithm', 'problem', 'learning', 'datum', 'function', 'university', 'introduction', 'task']
Topic 8: ['network', 'model', '

Iter [50]===
Topic 0: ['model', 'task', 'learning', 'deep', 'learn', 'network', 'neural', 'image', 'train', 'method']
Topic 1: ['datum', 'algorithm', 'learning', 'learn', 'problem', 'sample', 'method', 'set', 'training', 'class']
Topic 2: ['information', 'university', 'brain', 'analysis', 'datum', 'memory', 'structure', 'department', 'neural', 'e']
Topic 3: ['model', 'distribution', 'datum', 'gaussian', 'kernel', 'gan', 'generative', 'process', 'university', 'variable']
Topic 4: ['network', 'neural', 'input', 'learn', 'unit', 'output', 'model', 'training', 'weight', 'state']
Topic 5: ['neuron', 'network', 'model', 'neural', 'system', 'time', 'cell', 't', 'pattern', 'signal']
Topic 6: ['x', 'n', 'function', 'problem', 'algorithm', 'k', 'method', 'matrix', 't', 'optimization']
Topic 7: ['algorithm', 'problem', 'learning', 'policy', 'reinforcement', 'function', 'state', 'agent', 'learn', 'action']
Topic 8: ['datum', 'problem', 'cluster', 'network', 'user', 'algorithm', 'graph', 'clusterin

Iter [100]===
Topic 0: ['model', 'deep', 'task', 'learning', 'network', 'learn', 'image', 'neural', 'train', 'representation']
Topic 1: ['datum', 'learning', 'algorithm', 'problem', 'learn', 'sample', 'method', 'training', 'function', 'label']
Topic 2: ['university', 'brain', 'neural', 'information', 'structure', 'department', 'method', 'analysis', 'datum', 'approach']
Topic 3: ['model', 'distribution', 'inference', 'datum', 'gaussian', 'variable', 'process', 'gan', 'sample', 'generative']
Topic 4: ['network', 'neural', 'input', 'output', 'learn', 'unit', 'weight', 'training', 'system', 'error']
Topic 5: ['neuron', 'network', 'neural', 'model', 'system', 'cell', 'time', 'dynamic', 'analog', 'activity']
Topic 6: ['x', 'problem', 'n', 'algorithm', 'function', 'k', 't', 'gradient', 'optimization', 'matrix']
Topic 7: ['learning', 'algorithm', 'problem', 'policy', 'reinforcement', 'learn', 'agent', 'function', 'action', 'state']
Topic 8: ['datum', 'graph', 'cluster', 'node', 'algorithm', 'u

Iter [150]===
Topic 0: ['network', 'model', 'task', 'deep', 'learning', 'learn', 'neural', 'image', 'datum', 'train']
Topic 1: ['datum', 'learning', 'algorithm', 'problem', 'learn', 'sample', 'method', 'set', 'label', 'example']
Topic 2: ['university', 'brain', 'neural', 'model', 'information', 'department', 'analysis', 'approach', 'datum', 'correlation']
Topic 3: ['model', 'distribution', 'inference', 'gaussian', 'process', 'datum', 'gan', 'sample', 'variable', 'generative']
Topic 4: ['network', 'neural', 'input', 'learn', 'output', 'unit', 'weight', 'training', 'function', 'algorithm']
Topic 5: ['neuron', 'network', 'neural', 'model', 'system', 'cell', 'time', 'dynamic', 'analog', 'simulation']
Topic 6: ['x', 'problem', 'n', 'algorithm', 'function', 't', 'optimization', 'k', 'matrix', 'method']
Topic 7: ['learning', 'algorithm', 'policy', 'reinforcement', 'learn', 'problem', 'state', 'agent', 'action', 'game']
Topic 8: ['datum', 'graph', 'node', 'cluster', 'algorithm', 'problem', 'cl

Iter [200]===
Topic 0: ['network', 'deep', 'task', 'model', 'learning', 'learn', 'neural', 'image', 'representation', 'method']
Topic 1: ['datum', 'learning', 'problem', 'learn', 'algorithm', 'set', 'training', 'method', 'sample', 'label']
Topic 2: ['university', 'model', 'brain', 'neural', 'analysis', 'department', 'datum', 'information', 'edu', 'structure']
Topic 3: ['model', 'distribution', 'inference', 'gaussian', 'datum', 'sample', 'gan', 'generative', 'process', 'variational']
Topic 4: ['network', 'neural', 'input', 'unit', 'weight', 'learn', 'output', 'function', 'training', 'algorithm']
Topic 5: ['neuron', 'network', 'model', 'neural', 'system', 'cell', 'analog', 'abstract', 'control', 'time']
Topic 6: ['x', 'n', 'problem', 'function', 'algorithm', 'optimization', 't', 'k', 'matrix', 'gradient']
Topic 7: ['learning', 'policy', 'algorithm', 'reinforcement', 'problem', 'learn', 'agent', 'action', 'reward', 'control']
Topic 8: ['algorithm', 'datum', 'graph', 'cluster', 'problem', 

Iter [250]===
Topic 0: ['network', 'deep', 'model', 'task', 'learn', 'learning', 'neural', 'image', 'method', 'representation']
Topic 1: ['datum', 'learning', 'problem', 'learn', 'algorithm', 'sample', 'label', 'method', 'example', 'training']
Topic 2: ['university', 'model', 'neural', 'brain', 'analysis', 'department', 'science', 'information', 'method', 'time']
Topic 3: ['model', 'distribution', 'inference', 'datum', 'process', 'gaussian', 'sample', 'generative', 'gan', 'variable']
Topic 4: ['network', 'neural', 'input', 'unit', 'weight', 'learn', 'output', 'function', 'training', 'state']
Topic 5: ['neuron', 'network', 'model', 'neural', 'system', 'cell', 'analog', 'study', 'pattern', 'activity']
Topic 6: ['x', 'problem', 'n', 'function', 'algorithm', 't', 'k', 'optimization', 'matrix', 'method']
Topic 7: ['learning', 'algorithm', 'policy', 'reinforcement', 'problem', 'learn', 'agent', 'game', 'state', 'action']
Topic 8: ['algorithm', 'datum', 'graph', 'problem', 'cluster', 'node', 